In [14]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import json
from pandas.io.json import json_normalize

In [15]:
# use json_normalize to make the json readable
with open('data/rating.json') as file: 
    dog_file = json.load(file) 
dog_df = json_normalize(dog_file['dog_breeds'])
dog_df.head(10)

,Affenpinscher. Adaptability,Affenpinscher. All Around Friendliness,Affenpinscher. Exercise Needs,Affenpinscher. Health Grooming,Affenpinscher. Trainability,Affenpinscher.Adapts Well to Apartment Living,Affenpinscher.Affectionate with Family,Affenpinscher.Amount Of Shedding,Affenpinscher.Dog Friendly,Affenpinscher.Drooling Potential,...,Yorkshire Terrier.Potential For Playfulness,Yorkshire Terrier.Potential For Weight Gain,Yorkshire Terrier.Prey Drive,Yorkshire Terrier.Sensitivity Level,Yorkshire Terrier.Size,Yorkshire Terrier.Tendency To Bark Or Howl,Yorkshire Terrier.Tolerates Being Alone,Yorkshire Terrier.Tolerates Cold Weather,Yorkshire Terrier.Tolerates Hot Weather,Yorkshire Terrier.Wanderlust Potential
0,3,3,4,2,3,5,5,1,4,1,...,5,2,2,5,1,3,2,2,2,3


In [16]:
# clean up columns by separating breeds and traits
breeds = dog_df.columns.str.split('.').str[0]
traits = dog_df.columns.str.split('.').str[1]

#drop duplicates to establish the number of unique values for each axis
breeds = breeds.drop_duplicates()
traits = traits.drop_duplicates()

#determine how many breeds there are and how many traits are measured (214 breeds and 31 traits)
len(breeds), len(traits)

(214, 31)

In [17]:
#resize the shape of the values to fit the new dataframe
dog_array = dog_df.values
dog_array.resize(len(breeds),len(traits))
dog_array.shape

new_dog_df = pd.DataFrame(dog_array, index=breeds, columns=traits)
#return the new dataframe
new_dog_df

,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,Drooling Potential,...,Potential For Playfulness,Potential For Weight Gain,Prey Drive,Sensitivity Level,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential
Affenpinscher,3,3,4,2,3,5,5,1,4,1,...,4,3,3,3,1,2,1,3,3,2
Afghan Hound,4,4,4,2,3,5,5,4,4,1,...,4,1,5,5,4,2,2,5,5,5
Airedale Terrier,2,4,5,3,5,1,4,2,4,1,...,5,4,5,3,3,4,2,3,3,4
Akita,3,2,4,4,4,2,5,5,1,5,...,5,4,4,5,4,5,1,5,2,4
Alaskan Klee Kai,3,3,4,3,4,3,4,4,2,2,...,4,2,5,4,2,3,2,5,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Highland White Terrier,4,4,4,3,4,4,5,4,5,1,...,4,4,4,4,2,4,2,4,3,3
Whippet,3,5,4,3,4,5,5,2,4,1,...,5,1,5,5,3,1,2,1,4,4
Wirehaired Pointing Griffon,3,5,4,3,4,1,5,1,4,1,...,5,3,4,4,3,4,1,4,3,4
Yorkipoo,4,4,4,2,4,5,5,1,3,1,...,4,2,3,4,1,5,3,2,3,2


In [18]:
new_dog_df.columns

Index([' Adaptability', ' All Around Friendliness', ' Exercise Needs',
       ' Health Grooming', ' Trainability', 'Adapts Well to Apartment Living',
       'Affectionate with Family', 'Amount Of Shedding', 'Dog Friendly',
       'Drooling Potential', 'Easy To Groom', 'Easy To Train', 'Energy Level',
       'Exercise Needs', 'Friendly Toward Strangers', 'General Health',
       'Good For Novice Owners', 'Incredibly Kid Friendly Dogs',
       'Intelligence', 'Intensity', 'Potential For Mouthiness',
       'Potential For Playfulness', 'Potential For Weight Gain', 'Prey Drive',
       'Sensitivity Level', 'Size', 'Tendency To Bark Or Howl',
       'Tolerates Being Alone', 'Tolerates Cold Weather',
       'Tolerates Hot Weather', 'Wanderlust Potential'],
      dtype='object')

In [19]:
#narrow list of traits to most relevant subset
new_dog_df = new_dog_df[[' All Around Friendliness', 'Energy Level','Exercise Needs', 'Friendly Toward Strangers', 'Incredibly Kid Friendly Dogs', 'Intelligence', 'Size']]

new_dog_df

,All Around Friendliness,Energy Level,Exercise Needs,Friendly Toward Strangers,Incredibly Kid Friendly Dogs,Intelligence,Size
Affenpinscher,3,4,3,3,1,4,1
Afghan Hound,4,5,4,2,5,4,4
Airedale Terrier,4,5,5,3,4,5,3
Akita,2,4,4,2,1,3,4
Alaskan Klee Kai,3,4,4,2,3,4,2
...,...,...,...,...,...,...,...
West Highland White Terrier,4,4,4,3,4,4,2
Whippet,5,5,5,4,5,4,3
Wirehaired Pointing Griffon,5,5,4,5,5,5,3
Yorkipoo,4,5,3,2,4,4,1


In [20]:
#rename and reorganize columns
new_dog_df = new_dog_df.rename(columns={' All Around Friendliness': 'friendly_overall', 'Energy Level': 'energy', 'Exercise Needs': 'exercise_needs', 'Friendly Toward Strangers': 'friendly_strangers', 'Incredibly Kid Friendly Dogs': 'friendly_kids', 'Intelligence': 'intelligence', 'Size': 'size'})

new_dog_df = new_dog_df[['size', 'intelligence', 'energy', 'exercise_needs', 'friendly_overall', 'friendly_kids', 'friendly_strangers']]

new_dog_df

,size,intelligence,energy,exercise_needs,friendly_overall,friendly_kids,friendly_strangers
Affenpinscher,1,4,4,3,3,1,3
Afghan Hound,4,4,5,4,4,5,2
Airedale Terrier,3,5,5,5,4,4,3
Akita,4,3,4,4,2,1,2
Alaskan Klee Kai,2,4,4,4,3,3,2
...,...,...,...,...,...,...,...
West Highland White Terrier,2,4,4,4,4,4,3
Whippet,3,4,5,5,5,5,4
Wirehaired Pointing Griffon,3,5,5,4,5,5,5
Yorkipoo,1,4,5,3,4,4,2


In [21]:
breeds.values

array(['Affenpinscher', 'Afghan Hound', 'Airedale Terrier', 'Akita',
       'Alaskan Klee Kai', 'Alaskan Malamute', 'American Bulldog',
       'American English Coonhound', 'American Eskimo Dog',
       'American Foxhound', 'American Pit Bull Terrier',
       'American Staffordshire Terrier', 'American Water Spaniel',
       'Anatolian Shepherd Dog', 'Appenzeller Sennenhunde',
       'Australian Cattle Dog', 'Australian Kelpie',
       'Australian Shepherd', 'Australian Terrier', 'Azawakh', 'Barbet',
       'Basenji', 'Basset Hound', 'Beagle', 'Bearded Collie',
       'Bedlington Terrier', 'Belgian Malinois', 'Belgian Sheepdog',
       'Belgian Tervuren', 'Berger Picard', 'Bernedoodle',
       'Bernese Mountain Dog', 'Bichon Frise', 'Black Mouth Cur',
       'Black Russian Terrier', 'Black and Tan Coonhound', 'Bloodhound',
       'Blue Lacy', 'Bluetick Coonhound', 'Boerboel', 'Bolognese',
       'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Terrier',
       'Bouvier des Flandres

In [26]:
rds_connection_string = "<postgres>:<postgres>@localhost:5432/nyc_dogs_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [27]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "<postgres>"

(Background on this error at: http://sqlalche.me/e/e3q8)